### GPT-2 모델로 텍스트 생성하기 목차
* [Chapter 1 마스크 멀티 헤드 어텐션)](#chapter1)


### Chapter 1 마스크 멀티 헤드 어텐션 <a class="anchor" id="chapter1"></a>
1. 트랜스포머 디코더(Transformer Decoder)는 원본 트랜스포머 모델에서 인코더가 이해한 내용을 바탕으로 결과를 생성하는 역활을 한다.
   - 영어 문장을 한글로 변역한다면 디코더는 인코더가 이해한 문장의 의미를 바탕으로 번역한다.
   - 각 단어를 예측할 때 지속적으로 이전 단어들과의 관계를 반영하기 때문에 사람이 쓴 것 처럼 높은 품질의 텍스트를 생성 할 수 있습니다.

2. 인코더-디코더 구조에서 디코더만 따로 분리해서 훈련할 수 있다.
    - 프롬프트가 입력되면 프롬프트 다음에 등장할 토큰을 생성하는 역활을 한다.
    - 훈련할 때의 타깃은 입력되는 텍스트 데이터의 다음 토큰이 된다.

3. 'Stay hungry, stay folish'라는 문장이 있다면, 'Stay'를 입력하면 'hungry'를 예측하는 식이다.
    - 이 때 디코더는 이전에 등장한 단어들만을 참고해서 다음 단어를 예측해야 한다.
    - 즉, 'stay'를 입력했을 때 'hungry'가 아니라 'folish'가 다시 예측되는 일이 없어야 한다.
    - 이를 위해 디코더는 마스크 멀티 헤드 어텐션(Masked Multi-Head Attention)이라는 기법을 사용한다.
    - 첫 번째 단어부터 토큰을 하나씩 이동하면서 다음 토큰을 예측하는 과정으로 쉽게 확장할 수 있다.
    - 디코더 모델이 'stay' 토큰을 사용해 'thund'를 예측했다. 하지만 정답은 'hungry'이다.
       - 두 단의 차이가 손실이 되고, 이 손실을 통해 모델의 가중치를 훈련한다.
       - 다음 등장할 토큰을 예측하기 위해 입력에 있는 'hungry' 토큰을 사용한다면 속임수를 쓰는 것과 같다.
       - 모델은 'stay' 토큰만을 바라봐야한다.
    - 입력 문장에서 하나의 토큰을 더 사용해 'stay hungry'가 입력되면 디코더 모델은 공백을 예측한다.
       - 정답 토큰은 ','이다.
       - 모델이 이전 단계에서 'thund'를 예측했지만 모델의 입력으로 사용하지 않았다.
       - 모델은 원복 텍스트에 있는 'stay hungry'를 입력으로 사용했다.
    - 마지막으로 하나의 토크을 더 사용해 'stay hungry,'를 입력으로 받았다.
       - 정답과 동이한 'stay'를 예측했다.       

        ![예측](image/05-01-predict2.png)   

4. 텍스트 생성 모델인 트랜스포머 디코더 모델은 입력 텍스트의 다음 토큰을 타깃으로 사용할 수 있다.
   - 이런한 훈련 방식을 자기지도 학습(self-supervised learning) 또는 코잘 언어 모델링(causal language modeling)이라고 한다.
   - 자기지도 학습 방식을 사용하면 훈련 데이터를 레이블링하는 수고를 들이지 않고도 많은 텍스트 데이터로부터 대규모 언어 모델을 훈련할 수 있다.
   - 여기서 핵심은 모델이 훈련 과정 중에 입력에 있는 다음 토큰을 훔쳐봐서는 않된다는 것이다.
   - 이를 위해 디코더는 마스크 멀티 헤드 어텐션(Masked Multi-Head Attention)이라는 기법을 사용한다.

5. 마스크 멀티 헤드 어텐션(Masked Multi-Head Attention)
    - 어텐션 점수를 계산할 때 현재 토큰에서 미래 토큰을 바라보지 못하도록 마스킹해 학습을 제한한다.
    - 5 * 5 크기의 어텐션 행렬에서 주 대각선 윗부분의 점수를 가린다.
        - 'hungry' 토큰을 처리할 때 다음에 나오는 ','와 'stay', 'foolish' 토큰에 대한 점수를 사용할 수 없다.
        - 어텐션 행렬은 셀프 어텐션 계산식의 쿼리와 키를 곱한 결과로, (n_tokens, n_tokens) 크기를 가진다.
        - 주 대간선은 왼쪽에서 오른쪽 아래까지 역슬래시(\) 모양으로 이어지는 대각선을 말한다.
        - MultiheadAttention 클래스의 attention_mask 매개변수에 마스킹 정보를 전달하기만 하면 자동으로 마스크 멀티해드 어텐션이 실행된다.
           - 이 마스킹을 종종 코잘 마스킹(causal masking)이라고 부른다.

        ![마스킹](image/05-01-masking2.png)   


In [5]:
import keras
from keras import layers
import keras

def make_causal_mask(seq_len):
    # keras.ops.arange() 함수를 사용하여 0부터 시퀸스 길이까지 채워진 텐서를 만든다.
    #   - 입력 값이 5인경우 [0, 1, 2, 3, 4] 형태의 텐서가 만들어진다.
    n_hori = keras.ops.arange(seq_len)
    
    # keras.ops.expand_dims() 함수를 사용하여 n_hori 텐서의 마지막 차원을 확장한다.
    #   - 입력 값이 [0, 1, 2, 3, 4] 인 경우 [[0], [1], [2], [3], [4]] 형태의 텐서가 만들어진다.
    #   - (seq_len, ) 형태의 텐서를 (seq_len, 1) 형태로 바꾼다.
    n_vert = keras.ops.expand_dims(n_hori, axis=-1)
    
    # n_vert >= n_hori 비교 연산을 수행하여 인덱스가 같거나 큰 위치에 True 값을 채운다.
    mask = n_vert >= n_hori
    return mask

   - expand_dims() 함수를 통해 2차원 텐서로 바뀐 텐서는 다음과 같다.
      - 입력 값이 [0, 1, 2, 3, 4] 인 경우 [[0], [1], [2], [3], [4]] 형태의 텐서가 만들어진다.

      ![마스킹3](image/05-01-masking3.png)   

   - 다시 비교 연산을 통해 다음과 같은 결과를 얻게된다.
      - mask = n_vert >= n_hori

      ![마스킹4](image/05-01-masking4.png)      

In [6]:
causal_mask = make_causal_mask(5)
print(causal_mask)

I0000 00:00:1759660503.383198    1426 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5555 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:08:00.0, compute capability: 8.6


tf.Tensor(
[[ True False False False False]
 [ True  True False False False]
 [ True  True  True False False]
 [ True  True  True  True False]
 [ True  True  True  True  True]], shape=(5, 5), dtype=bool)


In [7]:
padding_make = [1, 1, 1, 0, 0]
keras.ops.minimum(causal_mask, padding_make)

<tf.Tensor: shape=(5, 5), dtype=int32, numpy=
array([[1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 1, 1, 0, 0],
       [1, 1, 1, 0, 0],
       [1, 1, 1, 0, 0]], dtype=int32)>

- 결과 그림으로 표현

    ![마스킹5](image/05-01-masking5.png)      

In [8]:
def make_attention_mask(padding_mask):
    # padding_mask의 크기가 (2, 5) 라고 가정
    batch_size, seq_len = keras.ops.shape(padding_mask)
    # causal_mask의 크기는 (5, 5)
    causal_mask = make_causal_mask(seq_len)
    # 배치 차원을 추가해 (2, 5, 5) 크기로 만든다.
    causal_mask = keras.ops.broadcast_to(causal_mask, (batch_size, seq_len, seq_len))
    # 브로드캐스팅을 위해 padding_mask의 크기를 (2, 1, 5)로 바꾼다.
    padding_mask = keras.ops.expand_dims(padding_mask, axis=1)
    return keras.ops.minimum(causal_mask, padding_mask)

In [ ]:
# 첫 번재 코잘 마스킹 행렬은 첫 번재 핑딩 마스크를 따라서 두 번째 열 이후가 모두 마스킹
# 두 번째 코잘 마스킹 행렬은 두 번째 핑딩 마스크를 따라서 네 번째 열 이후가 모두 마스킹
make_attention_mask([[1, 1, 0, 0, 0], [1, 1, 1, 1, 0]])

<tf.Tensor: shape=(2, 5, 5), dtype=int32, numpy=
array([[[1, 0, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0]],

       [[1, 0, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 1, 0, 0],
        [1, 1, 1, 1, 0],
        [1, 1, 1, 1, 0]]], dtype=int32)>